In [1]:
from importlib import reload
import sys

src_path =   '../src' # change as needed
sys.path.insert(0,src_path)

max_length = 128

train = True
debias = False

In [2]:
import data_generator;reload(data_generator)

train_data, val_data, test_data = data_generator.GetData(max_length)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1'.
INFO:absl:Downloading https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1: 161.62MB
INFO:absl:Downloading https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1: 321.62MB
INFO:absl:Downloaded https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1, Total size: 417.36MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1'.
100%|██████████| 397080/397080 [02:01<00:00, 3258.75it/s]


        tag  cat  occurences
0    B-MISC    0         800
1     I-LOC    1      328160
2    I-MISC    2      139600
3     I-ORG    3      237760
4     I-PER    4      977800
5         O    5     6661880
6  [nerCLS]    6      397080
7  [nerPAD]    7    39623065
8  [nerSEP]    8      397080
9    [nerX]    9     2063015

                tag  cat  occurences
0  AFRICAN-AMERICAN    0     4173000
1          EUROPEAN    1     4173000
2         [raceCLS]    2      397080
3         [racePAD]    3    39623065
4         [raceSEP]    4      397080
5           [raceX]    5     2063015

           tag  cat  occurences
0       FEMALE    0     4173000
1         MALE    1     4173000
2  [genderCLS]    2      397080
3  [genderPAD]    3    39623065
4  [genderSEP]    4      397080
5    [genderX]    5     2063015



In [3]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import model_utils; reload(model_utils)
    
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

model = model_utils.NER(max_length)
    
model.generate(bert_train_layers=4)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, None, 768)    108931396   input_ids[0][0]                  
                                                                 input_masks[0][0]            

In [4]:
if train:

    model.fit(
        sess,
        train_data,
        val_data,
        epochs=8,
        batch_size=32,
        debias=debias
    ) 
    
else: 
    model_utils.initialize_vars(sess)


acc_ner: 0.99; acc_ner_non_other: 0.97;  acc_gender: 0.50; acc_race: 0.50



acc_ner: 0.99; acc_ner_non_other: 0.97;  acc_gender: 0.50; acc_race: 0.50



acc_ner: 0.99; acc_ner_non_other: 0.97;  acc_gender: 0.50; acc_race: 0.50



acc_ner: 0.99; acc_ner_non_other: 0.97;  acc_gender: 0.50; acc_race: 0.50



acc_ner: 0.99; acc_ner_non_other: 0.98;  acc_gender: 0.50; acc_race: 0.50



acc_ner: 0.99; acc_ner_non_other: 0.98;  acc_gender: 0.50; acc_race: 0.50



acc_ner: 0.99; acc_ner_non_other: 0.98;  acc_gender: 0.50; acc_race: 0.50



acc_ner: 0.99; acc_ner_non_other: 0.98;  acc_gender: 0.50; acc_race: 0.50



In [5]:
bias = model.getBiasedPValues(test_data, num_iterations=10000)

In [6]:
bias

{'race': 0.0,
 'gender': 0.2807,
 'race_test_statistic': 0.3842757336007788,
 'gender_test_statistic': -0.05742955995490251,
 'race_std': 0.09622614605055416,
 'gender_std': 0.09672796008075966,
 'race_effect_size': 1.1134265250874718,
 'gender_effect_size': -0.16640029485785918}

In [7]:
sess.close()